This code tests whether the implemented function is actually working properly or not

It does this by calculating the moments over a unit cube with vertices (0,0,0) and (1,1,1) and edges aligned with axes and comparing with the analytical exact solution of the moments which is:

1/((i+1)*(j+1)*(k+1))

In [1]:
import pymeshlab
import numpy as np

In [2]:
# Create a cube meshed with triangles
m = pymeshlab.Mesh(
    vertex_matrix = [
        [0, 0, 0],
        [0, 0, 1],
        [0, 1 ,0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1 ,0],
        [1, 1 ,1]
    ],
    face_matrix = [
        [0,2,4],
        [4,2,6],
        [0,1,3],
        [0,3,2],
        [0,5,1],
        [0,4,5],
        [1,7,3],
        [1,5,7],
        [4,7,5],
        [4,6,7],
        [7,6,2],
        [7,2,3]
    ])

In [3]:
v_matrix = m.vertex_matrix()

In [4]:
f_matrix = m.face_matrix()

In [5]:
np.array(f_matrix)+1

array([[1, 3, 5],
       [5, 3, 7],
       [1, 2, 4],
       [1, 4, 3],
       [1, 6, 2],
       [1, 5, 6],
       [2, 8, 4],
       [2, 6, 8],
       [5, 8, 6],
       [5, 7, 8],
       [8, 7, 3],
       [8, 3, 4]], dtype=int32)

By default faces are defined and stored in the counter-clockwise order, so should not have any issues calculating the moments

In [6]:
# Define the maximum order of moments to be calculated
max_m = 10

In [7]:
def C_ijk(C,i,j,k):
    return (C[0]**i)*(C[1]**j)*(C[2]**k)*np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))
        

In [8]:
def D_ijk(D_tensor,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # D_ijk=0
        pass
    elif (0==i) and (0==j) and (0==k):
        # D_ijk = 1
        D_tensor[i,j,k] = 1
    else:
        D_tensor[i,j,k] = B[0]*D_tensor[i-1,j,k]+B[1]*D_tensor[i,j-1,k]+B[2]*D_tensor[i,j,k-1]+C_ijk(C,i,j,k)
    

In [9]:
def S_ijk(S_tensor,D_tensor,A,B,C,i,j,k):
    if (i<0) or (j<0) or (k<0):
        # S_ijk = 0
        pass
    elif (0==i) and (0==j) and (0==k):
        # S_ijk = 1
        S_tensor[i,j,k] = 1
    else:
        S_tensor[i,j,k] = A[0]*S_tensor[i-1,j,k]+A[1]*S_tensor[i,j-1,k]+A[2]*S_tensor[i,j,k-1]+D_tensor[i,j,k]
    

In [10]:
def vol(A,B,C):
    return 1./6*np.linalg.det([A,B,C])


In [11]:
def M_ijk(f_matrix, v_matrix, max_m):
    M_tensor = np.zeros([max_m,max_m,max_m])
    
    
    for face_num in range(len(f_matrix)):
        
        
        # Get the three coordinates A,B,C
        # Any mesh library should return these in CCW order
        [A,B,C] = v_matrix[f_matrix[face_num]]
        
        # Calculate the face moments tensor
        # Each face moments tensor requires calculating S_ijk, D_ijk, and C_ijk Tensor Calculations.

        # C_ijk tensor is not reused, calculate on the fly
        
        # D_ijk tensor is reused, calculate and store
        D_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        D_ijk(D_tensor,B,C,i,j,k)
                        
        # S_ijk tensor is reqused, calculate and store
        S_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        S_ijk(S_tensor,D_tensor,A,B,C,i,j,k)
                        
        # Now calculate the moments on current face
        f_M_tensor = np.zeros([max_m,max_m,max_m])
        for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        f_M_tensor[i,j,k] = (np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3)*np.linalg.det([A,B,C])*S_tensor[i,j,k]
        
        # Add contribution to the total moments
        M_tensor = M_tensor+f_M_tensor
                
    return M_tensor



In [12]:
def M_ijk_fast(f_matrix,v_matrix,max_m):
    num_faces = len(f_matrix)
    
    # Extract coordinates
    A = np.zeros([num_faces,3])
    B = np.zeros([num_faces,3])
    C = np.zeros([num_faces,3])
    for face_num in range(num_faces):
        [A[face_num,:],B[face_num,:],C[face_num,:]] = v_matrix[f_matrix[face_num]]
    # Calculate Determinants
    dets = np.linalg.det([[A[i],B[i],C[i]] for i in range(len(f_matrix))])
    
    # Allocate Tensors
    M_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    C_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    D_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    S_tensor = np.zeros([num_faces,max_m,max_m,max_m])
    
    # Calculate C Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        C_tensor[:,i,j,k] = (C[:,0]**i)*(C[:,1]**j)*(C[:,2]**k)*(np.math.factorial(i+j+k)/(np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k)))
                        
    # Calculate D Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # D_ijk=0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # D_ijk = 1
                            D_tensor[:,i,j,k] = 1
                        else:
                            D_tensor[:,i,j,k] = B[:,0]*D_tensor[:,i-1,j,k]+B[:,1]*D_tensor[:,i,j-1,k]+B[:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
    # Calculate S Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # S_ijk = 0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # S_ijk = 1
                            S_tensor[:,i,j,k] = 1
                        else:
                            S_tensor[:,i,j,k] = A[:,0]*S_tensor[:,i-1,j,k]+A[:,1]*S_tensor[:,i,j-1,k]+A[:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
    # Calculate M Tensor, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_tensor[:,i,j,k] = ((np.math.factorial(i)*np.math.factorial(j)*np.math.factorial(k))/np.math.factorial(i+j+k+3))*dets[:]*S_tensor[:,i,j,k]
    return np.sum(M_tensor,axis=0)

In [13]:


import torch
from math import factorial

def M_ijk_torch(f_matrix,v_matrix,max_m):
    num_faces = len(f_matrix)
    
    # Extract coordinates
    ABC = v_matrix[f_matrix]
    # Calculate Determinants
    dets = torch.det(ABC)
    
    # Allocate Tensors
    M_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float64)
    C_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float64)
    D_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float64)
    S_tensor = torch.zeros([num_faces,max_m,max_m,max_m],dtype=torch.float64)
    
    # Calculate C Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        C_tensor[:,i,j,k] = (ABC[:,2][:,0]**i)*(ABC[:,2][:,1]**j)*(ABC[:,2][:,2]**k)*(factorial(i+j+k)/(factorial(i)*factorial(j)*factorial(k)))
                        
    # Calculate D Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # D_ijk=0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # D_ijk = 1
                            D_tensor[:,i,j,k] = 1
                        else:
                            D_tensor[:,i,j,k] = ABC[:,1][:,0]*D_tensor[:,i-1,j,k]+ABC[:,1][:,1]*D_tensor[:,i,j-1,k]+ABC[:,1][:,2]*D_tensor[:,i,j,k-1]+C_tensor[:,i,j,k]
    
    
    # Calculate S Tensor, parallellized over faces
    for i in range(max_m):
            for j in range(max_m):
                for k in range(max_m):
                    if (i+j+k)<=max_m:
                        if (i<0) or (j<0) or (k<0):
                            # S_ijk = 0
                            pass
                        elif (0==i) and (0==j) and (0==k):
                            # S_ijk = 1
                            S_tensor[:,i,j,k] = 1
                        else:
                            S_tensor[:,i,j,k] = ABC[:,0][:,0]*S_tensor[:,i-1,j,k]+ABC[:,0][:,1]*S_tensor[:,i,j-1,k]+ABC[:,0][:,2]*S_tensor[:,i,j,k-1]+D_tensor[:,i,j,k]
        
    # Calculate M Tensor, parallellized over faces
    for i in range(max_m):
        for j in range(max_m):
            for k in range(max_m):
                if (i+j+k)<=max_m:
                    M_tensor[:,i,j,k] = ((factorial(i)*factorial(j)*factorial(k))/factorial(i+j+k+3))*(dets[:]*S_tensor[:,i,j,k])
                    
    return torch.sum(M_tensor,0)
    

# M_ijk_fast_pytorch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),3)

In [14]:
test_tensor = np.zeros((max_m,max_m,max_m))
for i in range(max_m):
    for j in range(max_m):
        for k in range(max_m):
            if (i+j+k)<=max_m:
                test_tensor[i,j,k] = 1/((i+1)*(j+1)*(k+1))

In [15]:
moments_fast = M_ijk_fast(f_matrix,v_matrix,max_m)
moments_torch = M_ijk_torch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),max_m)
moments_fast_error = np.linalg.norm(test_tensor-moments_fast)
moments_torch_error = torch.linalg.norm(torch.tensor(test_tensor)-moments_torch)
print('Numpy implementation error:   {}\nPytorch implementation error: {}'.format(moments_fast_error,moments_torch_error))


Numpy implementation error:   1.8762299503193352e-16
Pytorch implementation error: 1.8762299503193352e-16


Since the value is ~ machine precision, the code is working properly!

In [16]:
print(moments_fast)

[[[1.         0.5        0.33333333 0.25       0.2        0.16666667
   0.14285714 0.125      0.11111111 0.1       ]
  [0.5        0.25       0.16666667 0.125      0.1        0.08333333
   0.07142857 0.0625     0.05555556 0.05      ]
  [0.33333333 0.16666667 0.11111111 0.08333333 0.06666667 0.05555556
   0.04761905 0.04166667 0.03703704 0.        ]
  [0.25       0.125      0.08333333 0.0625     0.05       0.04166667
   0.03571429 0.03125    0.         0.        ]
  [0.2        0.1        0.06666667 0.05       0.04       0.03333333
   0.02857143 0.         0.         0.        ]
  [0.16666667 0.08333333 0.05555556 0.04166667 0.03333333 0.02777778
   0.         0.         0.         0.        ]
  [0.14285714 0.07142857 0.04761905 0.03571429 0.02857143 0.
   0.         0.         0.         0.        ]
  [0.125      0.0625     0.04166667 0.03125    0.         0.
   0.         0.         0.         0.        ]
  [0.11111111 0.05555556 0.03703704 0.         0.         0.
   0.         0.   

In [17]:
# # If interested in calculating the scaled and centered moments...

# v_matrix = np.array(v_matrix)

# # Calculate the centered and scaled moments
# v_matrix[:,0] = ( v_matrix[:,0] - moments[1,0,0]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
# v_matrix[:,1] = ( v_matrix[:,1] - moments[0,1,0]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
# v_matrix[:,2] = ( v_matrix[:,2] - moments[0,0,1]/moments[0,0,0] ) / np.cbrt(moments[0,0,0])
# moments = M_ijk(f_matrix,v_matrix,max_m)
# print(moments)

# # Note how running this cell twice gives nicer results!

Let's test what happens when we turn the cube inside out, i.e. change the order of the face vertices to be CW instead of CCW

In [18]:
v_matrix_CW = m.vertex_matrix()
f_matrix_CW = m.face_matrix()

In [19]:
f_matrix_CW[:,1] = f_matrix[:,2]
f_matrix_CW[:,2] = f_matrix[:,1]

In [20]:
moments_CW = M_ijk_torch(torch.tensor(f_matrix_CW,dtype=torch.long),torch.tensor(v_matrix_CW),max_m)

In [21]:
moments_CW[0,0,0]

tensor(-1.0000, dtype=torch.float64)

In [22]:
moments_torch[0,0,0]

tensor(1.0000, dtype=torch.float64)

Check performance

In [23]:
%timeit M_ijk_fast(f_matrix,v_matrix,max_m)

4.68 ms ± 54.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%timeit M_ijk_torch(torch.tensor(f_matrix,dtype=torch.long),torch.tensor(v_matrix),max_m)

50.3 ms ± 44.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
